In [1]:
import agate

text_type = agate.Text()

In [2]:
# Function to create a geoid, needed for 
def geoid2(row):
    geoid_string = str(row['GEOID2'])
    i=1
    if len(geoid_string) == 8:
        i=2
    geoid_string = geoid_string[i:]
    return '170%s%s' % (row['COUNTY'],geoid_string)

In [3]:
# Function to add up all people younger than 18 from 2010-14 ACS table B01001
def sum_child_2014(row):
    total = 0
    fields = ['HD01_VD03','HD01_VD04','HD01_VD05','HD01_VD06','HD01_VD27','HD01_VD28','HD01_VD29','HD01_VD30']
    for field in fields:
        total += row[field]
    return total

In [4]:
# Agate class to calculate the a percentage 
class PercentOfTotal(agate.Computation):
    def __init__(self, total_name, column_name):
        self._total = total_name
        self._column_name = column_name

    def get_computed_data_type(self, table):
        return agate.Number()

    def run(self, table):
        
        new_column = []

        for row in table.rows:
            
            total = row[self._total]
            column = row[self._column_name]
            
            if column and total:
                v = 100*(column/total)
                new_column.append(v)
            else:
                new_column.append(0)
            
        return new_column

In [5]:
# Agate class to sum all columns in a row
class SumColumns(agate.Computation):
    def __init__(self, columns):
        self._columns = columns

    def get_computed_data_type(self, table):
        return agate.Number()

    def run(self, table):
        c = self._columns

        new_column = []

        for row in table.rows:

            values = [row[x] for x in c]

            new_column.append(sum(values))

        return new_column

In [6]:
# Agate class to compute density
class Density(agate.Computation):
    def __init__(self, pop, area):
        self._pop = pop
        self._area = area

    def get_computed_data_type(self, table):
        return agate.Number()

    def run(self, table):

        new_column = []

        for row in table.rows:

            pop = row[self._pop]
            area = row[self._area]
            
            new_column.append(pop/area)

        return new_column

In [7]:
def analyze_2014():

    tester = agate.TypeTester(force={
        'GEO.id2': agate.Text()
    })

    age_2014 = agate.Table.from_csv('census_raw/aff_download_2014_age1/ACS_14_5YR_B01001_with_ann.csv',column_types=tester)
    age_2014_filter = age_2014.select(('GEO.id2','HD01_VD01','HD01_VD03','HD01_VD04','HD01_VD05','HD01_VD06','HD01_VD27','HD01_VD28','HD01_VD29','HD01_VD30'))
    census_lookup = agate.Table.from_csv('census_raw/modified/2010_Tract_to_Community_Area_Equivalency_File_modified.csv')
    
    families_2014_all = agate.Table.from_csv('census_raw/modified/acs_2014_families_all.csv',column_types=tester)
    families_2014_all_filter = families_2014_all.select(('GEO.id2','HD01_VD01','HD01_VD03','HD01_VD07'))
    
    families_2014 = agate.Table.from_csv('census_raw/modified/acs_2014_families.csv',column_types=tester)
    families_2014_filter = families_2014.select(('GEO.id2','HC01_EST_VC02','HC01_EST_VC10'))
    
    families_2014_all_rename = families_2014_all_filter.rename(column_names = ['GEO.id2','total_households','families_with_children','female_head_households'])
            
    census_lookup_geoid2 = census_lookup.compute([
        ('GEO.id2', agate.Formula(text_type, geoid2)),
    ])
    
    census_all = age_2014_filter.join(families_2014_all_rename, 'GEO.id2', 'GEO.id2', inner=True)
    
    age_2014_joined = census_all.join(census_lookup_geoid2, 'GEO.id2', 'GEO.id2', inner=True)

    age_2014_computed = age_2014_joined.compute([
        ('children_2014_total', agate.Formula(agate.Number(), sum_child_2014)),
        ('children_2014_0_4', SumColumns(['HD01_VD03','HD01_VD27'])),
        ('children_2014_5_9', SumColumns(['HD01_VD04','HD01_VD28'])),
        ('children_2014_10_14', SumColumns(['HD01_VD05','HD01_VD29'])),
        ('children_2014_15_17', SumColumns(['HD01_VD06','HD01_VD30'])),
    ])
    
    age_2014_group_ca = age_2014_computed.group_by('CHGOCA')

    age_2014_group_ca_counts = age_2014_group_ca.aggregate([
        ('total_2014', agate.Sum('HD01_VD01')),
        ('children_2014_total', agate.Sum('children_2014_total')),
        ('children_2014_0_4', agate.Sum('children_2014_0_4')),
        ('children_2014_5_9', agate.Sum('children_2014_5_9')),
        ('children_2014_10_14', agate.Sum('children_2014_10_14')),
        ('children_2014_15_17', agate.Sum('children_2014_15_17')),
        ('total_households_2014', agate.Sum('total_households')),
        ('total_families_with_kids',agate.Sum('families_with_children')),
    ])

    age_2014_group_ca_counts.print_table(max_rows=10)

    return age_2014_group_ca_counts

In [8]:
a2014 = analyze_2014()

|---------+------------+---------------------+-------------------+-------------------+---------------------+------|
|  CHGOCA | total_2014 | children_2014_total | children_2014_0_4 | children_2014_5_9 | children_2014_10_14 | ...  |
|---------+------------+---------------------+-------------------+-------------------+---------------------+------|
|       1 |     55,053 |              10,281 |             3,671 |             2,491 |               2,668 | ...  |
|       2 |     73,019 |              18,748 |             5,880 |             5,417 |               4,953 | ...  |
|      77 |     55,276 |               6,569 |             2,675 |             1,335 |               1,619 | ...  |
|       3 |     54,062 |               6,109 |             2,550 |             1,421 |               1,342 | ...  |
|       4 |     39,547 |               6,610 |             2,744 |             1,824 |               1,242 | ...  |
|       5 |     34,815 |               6,965 |             3,081 |      

In [9]:
age_join_percentage = a2014.compute([
    ('percent_child_2014', PercentOfTotal('total_2014', 'children_2014_total')),
    ('percent_child_2014_0_4', PercentOfTotal('total_2014', 'children_2014_0_4')),
    ('percent_child_2014_5_9', PercentOfTotal('total_2014', 'children_2014_5_9')),
    ('percent_child_2014_10_14', PercentOfTotal('total_2014', 'children_2014_10_14')),
    ('percent_child_2014_15_17', PercentOfTotal('total_2014', 'children_2014_15_17')),
    ('percent_child_2014_0_4_total', PercentOfTotal('children_2014_total', 'children_2014_0_4')),
    ('percent_child_2014_5_9_total', PercentOfTotal('children_2014_total', 'children_2014_5_9')),
    ('percent_child_2014_10_14_total', PercentOfTotal('children_2014_total', 'children_2014_10_14')),
    ('percent_child_2014_15_17_total', PercentOfTotal('children_2014_total', 'children_2014_15_17')),
    ('percent_families_2014_with_kids', PercentOfTotal('total_households_2014', 'total_families_with_kids')),
])

In [10]:
comm_areas = agate.Table.from_csv('raw_data/CommAreas.csv')
comm_areas_filter = comm_areas.select(['area_num_1','community','sq_miles'])

age_join_percentage_comm_name = comm_areas_filter.join(age_join_percentage,'area_num_1','CHGOCA')
age_join_percentage_density = age_join_percentage_comm_name.compute([
    ('density_pop_2014', Density('total_2014', 'sq_miles')),
    ('density_households_2014', Density('total_households_2014', 'sq_miles')),
    ('density_kids_2014', Density('children_2014_total', 'sq_miles')),
    ('density_fam_2014', Density('total_families_with_kids', 'sq_miles')),
    ('people_per_households_2014', Density('total_2014', 'total_households_2014')),
])

age_join_percentage_density.to_csv('census_analysis/age_change_density_2014.csv')

In [11]:
sorted_density = age_join_percentage_density.order_by('density_fam_2014', reverse=True)
sorted_density.print_bars('community','density_fam_2014')

community                            density_fam_2014
ALBANY PARK            3,267.271107482545482756879103 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░            
ROGERS PARK            2,833.529849415020209974895252 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                   
BELMONT CRAGIN         2,699.185338182650751788149598 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                     
HERMOSA                2,647.425515504028430750497247 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                      
GAGE PARK              2,495.572701182235489081755708 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                        
WEST RIDGE             2,331.576720260076167105395085 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                           
BRIGHTON PARK          2,318.645517230232463314744741 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                           
AVONDALE               2,298.216537578966528629132491 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░       